<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import folium
import pandas as pd
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [2]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

Now, you can take a look at what are the coordinates for each site.


In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [5]:
# Create a red circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle: `folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`
An example of folium.Marker: `folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`

In [6]:
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
# Initial the map
site_map = folium.Map(location = [31, -100], zoom_start = 4)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. 
# In addition, add Launch site name as a popup label
for latitude, longtitude, label in zip(launch_sites_df["Lat"], 
                                       launch_sites_df["Long"], 
                                       launch_sites_df["Launch Site"]):
    folium.Circle([latitude, longtitude], 
                  radius=50, 
                  color='blue', 
                  fill=True, 
                  popup = label
                 ).add_to(site_map)
    folium.map.Marker([latitude, longtitude], 
                      icon = DivIcon(icon_size=(60,20), 
                                     icon_anchor=(0,0), 
                                     html='<div style="font-size: 10pt; color:blue;"><b>%s</b></div>' % label)
                     ).add_to(site_map)

site_map

The launch sites are not located near the Equator line, however, all of them are in the south of the US, which is the closest option for this country. All launch sites are in very close proximity to the coast. 

Three of them (KSC LC-39A, CCAFS LC-40, and CCAFS SLC-40) are located on the coast with water to the east, which is beneficial in terms of safety and speed. 

The launches are usually eastward and located closer to the equator to take advantage of the velocity boost to rocket velocity from the rotation of the earth. In this case, it is desirable for the rocket path to be over uninhabited areas due to a crash chance. The sea is considered an uninhabited area at least of permanent buildings.

# Task 2: Mark the success/failed launches for each site on the map

Next, we try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
The data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not.

In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


We create markers for all launch records. If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

A launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [9]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

for index, outcome in enumerate(spacex_df["class"]):
    if outcome == 1:
        spacex_df.loc[index, "marker_color"] = "green"
    else:
        spacex_df.loc[index, "marker_color"] = "red"

spacex_df.tail()

,Launch Site,Lat,Long,class,marker_color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [10]:
# create a `MarkerCluster` object and add it to current site_map
marker_cluster = MarkerCluster().add_to(site_map)

# for each row in spacex_df data frame create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=...)

for latitude, longtitude, label, color in zip(spacex_df["Lat"],
                                              spacex_df["Long"],
                                              spacex_df["Launch Site"],
                                              spacex_df["marker_color"]):
    folium.Marker(location = [latitude, longtitude],
                  icon = folium.Icon(color = "white", icon_color = color),
                  popup = label
                 ).add_to(marker_cluster)

site_map

From the color-labeled markers in marker clusters, we see that the KSC LC-39A launch site have the highest success rate.

# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [11]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [12]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [13]:
# find coordinate of the closet coastline
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

launch_sites_df["Lat_sea"] = [28.56266, 28.56366, 28.60189, 34.63571]
launch_sites_df["Long_sea"] = [-80.56786, -80.56802, -80.58863, -120.62507]

for index, lat, lat_sea, long, long_sea in zip(launch_sites_df.index,
                                               launch_sites_df["Lat"],
                                               launch_sites_df["Lat_sea"],
                                               launch_sites_df["Long"], 
                                               launch_sites_df["Long_sea"]):
    
    launch_sites_df.loc[index, "Distance_sea"] = calculate_distance(lat, long, lat_sea, long_sea)

launch_sites_df

,Launch Site,Lat,Long,Lat_sea,Long_sea,Distance_sea
0,CCAFS LC-40,28.562302,-80.577356,28.56266,-80.56786,0.928592
1,CCAFS SLC-40,28.563197,-80.576820,28.56366,-80.56802,0.861231
2,KSC LC-39A,28.573255,-80.646895,28.60189,-80.58863,6.521461
3,VAFB SLC-4E,34.632834,-120.610745,34.63571,-120.62507,1.349433


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point

In [14]:
# Create and add a folium.Marker on your selected closest coastline point on the map

for lat, lat_point, long, long_point, lable in zip(launch_sites_df["Lat"],
                                               launch_sites_df["Lat_sea"],
                                               launch_sites_df["Long"],
                                               launch_sites_df["Long_sea"],
                                               launch_sites_df["Distance_sea"]):
        # Display the distance between coastline point and launch site using the icon property 
        folium.Marker(location = [lat_point, long_point],
                  icon = DivIcon(icon_size=(20,20),
                                 icon_anchor=(0,0),
                                 html='<div style="font-size: 10pt; color:blue;"><b>%s</b></div>' % "{:10.2f} KM".format(lable))
                  ).add_to(site_map)
        # Create a `folium.PolyLine` object using the point coordinates and launch site coordinate
        folium.PolyLine(locations = ([lat, long], [lat_point, long_point]), 
                        weight = 2
                        ).add_to(site_map)
site_map    


In [15]:
# find coordinate of the closet town
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

launch_sites_df["Lat_town"] = [28.61223, 28.61223, 28.61223, 34.63895]
launch_sites_df["Long_town"] = [-80.80793, -80.80793, -80.80793, -120.45792]

for index, lat, lat_town, long, long_town in zip(launch_sites_df.index,
                                                 launch_sites_df["Lat"],
                                                 launch_sites_df["Lat_town"],
                                                 launch_sites_df["Long"], 
                                                 launch_sites_df["Long_town"]):
    
    launch_sites_df.loc[index, "Distance_town"] = calculate_distance(lat, long, lat_town, long_town)

launch_sites_df

,Launch Site,Lat,Long,Lat_sea,Long_sea,Distance_sea,Lat_town,Long_town,Distance_town
0,CCAFS LC-40,28.562302,-80.577356,28.56266,-80.56786,0.928592,28.61223,-80.80793,23.194694
1,CCAFS SLC-40,28.563197,-80.576820,28.56366,-80.56802,0.861231,28.61223,-80.80793,23.221887
2,KSC LC-39A,28.573255,-80.646895,28.60189,-80.58863,6.521461,28.61223,-80.80793,16.313957
3,VAFB SLC-4E,34.632834,-120.610745,34.63571,-120.62507,1.349433,34.63895,-120.45792,14.002776


In [16]:
# Create and add a folium.Marker on your selected closest town on the map

# VAFB - town
lable = launch_sites_df.loc[3, "Distance_town"]
# Display the distance between coastline point and launch site using the icon property 
folium.Marker(location = [launch_sites_df.loc[3, "Lat_town"], launch_sites_df.loc[3, "Long_town"]],
              icon = DivIcon(icon_size=(20,20),
                             icon_anchor=(0,0),
                             html='<div style="font-size: 10pt; color:blue;"><b>%s</b></div>' % "{:10.2f} KM".format(lable))
              ).add_to(site_map)
# Create a `folium.PolyLine` object using the point coordinates and launch site coordinate
folium.PolyLine(locations = ([launch_sites_df.loc[3, "Lat"], launch_sites_df.loc[3, "Long"]], [launch_sites_df.loc[3, "Lat_town"], launch_sites_df.loc[3, "Long_town"]]), 
                weight = 2
                ).add_to(site_map)

# KSC - town
lable = launch_sites_df.loc[2, "Distance_town"]
# Display the distance between coastline point and launch site using the icon property 
folium.Marker(location = [launch_sites_df.loc[2, "Lat_town"], launch_sites_df.loc[2, "Long_town"]],
              icon = DivIcon(icon_size=(20,20),
                             icon_anchor=(0,0),
                             html='<div style="font-size: 10pt; color:blue;"><b>%s</b></div>' % "{:10.2f} KM".format(lable))
              ).add_to(site_map)
# Create a `folium.PolyLine` object using the point coordinates and launch site coordinate
folium.PolyLine(locations = ([launch_sites_df.loc[2, "Lat"], launch_sites_df.loc[2, "Long"]], [launch_sites_df.loc[2, "Lat_town"], launch_sites_df.loc[2, "Long_town"]]), 
                weight = 2
                ).add_to(site_map)

site_map    


All launch sites are in very close proximity to the coast. There are railways and highways in launch site close proximities (within 1 km), which is might be important for supply deliveries. And there is a certain distance away from cities.

KSC LC-39A site is about 6.5 km awya from a coast line, while others are just 1 km away.

Three of them (KSC LC-39A, CCAFS LC-40, and CCAFS SLC-40) are located on the coast with water to the east, which is beneficial in terms of safety and speed.

The launches are usually eastward and located closer to the equator to take advantage of the velocity boost to rocket velocity from the rotation of the earth. In this case, it is desirable for the rocket path to be over uninhabited areas due to a crash chance. The sea is considered an uninhabited area at least of permanent buildings.